In [1]:
from data.utils.lmdb_utils import LMDBDataset

dataset = LMDBDataset('data/graphs/HIV.lmdb')

In [7]:
all_atomic_nums = []
n = 20000
for i in range(20000):
    graph, target = dataset[i]
    all_atomic_nums.extend(graph['atom_atomic_nums'].tolist())

In [12]:
unique = set(all_atomic_nums)
print(len(unique))
for u in unique:
    print(u, all_atomic_nums.count(u))

53
0 7
2 5
4 54
5 368438
6 49195
7 70326
8 3181
10 213
11 2
12 2
13 213
14 1029
15 9080
16 4968
18 22
19 2
21 1
23 6
24 16
25 39
26 50
27 36
28 57
29 18
30 2
31 15
32 19
33 78
34 926
39 3
41 10
43 7
44 12
45 28
46 3
49 80
50 4
51 3
52 274
54 1
63 1
64 1
73 3
74 4
76 11
77 27
78 4
79 11
80 3
81 6
82 5
88 1
91 2
